# SAM: TP1 Accès aux données avec index 

Sujet pour étudiants

date de modification : 26/01/2023 16h

NOM: **KABONGO BUZANGU**

Prénom: **Ben**

Objectifs:
Savoir organiser des données en pages pour permettre de modifier un tuple en ne modifiant qu'une seule page.

Comprendre les méthodes d'accès suivantes :

*   Lecture séquentielle d'une fichier : "table access full"
*   Lecture d'un tuple dont on connait le rowid : "table access by index rowid"
*   Opération de sélection par lecture séquentielle et filtrage 

Comprendre les méthodes d'indexation :

*   Créer un index
*   Opération de Sélection par index et lecture par rowid

Mise à jour de données
*   Sélectionner un tuple et modifier un de ses attributs
*   Modifier l'index en conséquence lorsque l'attibut modifié est indexé

Persistence
*   Stocker un index (dans plusieurs pages) pour le reconstruire plus rapidement
*   Adapter en conséquence les opérations de modification de l'index


In [10]:
import os
import shutil as sh
import numpy as np
import random
from random import choice
from string import ascii_lowercase
import time
import tqdm

DATA = "data.csv"

# Générer un fichier

Création du fichier

In [2]:
# dure environ 40s pour 5M lignes

nb_lines = 5 * 1000 * 1000
# nb_lines = 100
nb_attributes = 7

longueur_attribut = 100
# string_val = "".join(choice(ascii_lowercase) for i in range(longueur_attribut))
long_string = ''.join('-' for i in range(longueur_attribut))

# a=[np.random.randint(0, int(nb_lines/(10**i)), nb_lines) for i in range(nb_attributes)]
nb_valeurs_distinctes = nb_lines

# le premier attribut est unique
a = [random.sample(range(nb_valeurs_distinctes), nb_lines)]

# les attributs suivants ont des domaines plus petits
for i in range(1, nb_attributes):
    nb_valeurs_distinctes = max(2, int(nb_valeurs_distinctes / 2))
    a.append(np.random.randint(0, nb_valeurs_distinctes, nb_lines))

b = [ ','.join(map(lambda x: str(x), e)) + f",{long_string}\n" for e in zip(*a)]

with open(DATA, "w") as f:
    f.write(''.join(b))

In [3]:
%%bash
echo "head : "
head -n 2 data.csv
echo "tail : "
tail -n 2 data.csv
echo "size (lines) :"
wc -l data.csv

head : 
3800875,197341,997143,58217,282194,127657,24795,----------------------------------------------------------------------------------------------------
832669,1178778,1045166,127448,239496,4692,64360,----------------------------------------------------------------------------------------------------
tail : 
2856992,1764808,837969,557654,167356,77205,50252,----------------------------------------------------------------------------------------------------
2220043,99233,814405,414236,22558,146605,17644,----------------------------------------------------------------------------------------------------
size (lines) :
 5000000 data.csv


# Lecture séquentielle

In [4]:
def filtrer_fichier(fichier, valeur_recherchee):
    with open(fichier, "r") as f:
        for i, line in enumerate(f):
            a = int(line.split(',')[0])
            if a == s :
                print(f"ligne {i} :", line.strip())


s = np.random.randint(nb_valeurs_distinctes)
print("valeur recherchée :", s)

t1 = time.time()
filtrer_fichier(DATA, s)
print("done in", time.time() - t1, "s")

valeur recherchée : 4564
ligne 96815 : 4564,473887,675094,287123,48563,132191,51086,----------------------------------------------------------------------------------------------------
done in 3.253901958465576 s


# Découper le fichier en pages

In [5]:
def page_dir_name(fichier):
    return fichier.split('.')[0] + "_pages"

def decoupe_fichier_en_pages(fichier, nb_tuple_par_page):
    page_dir = page_dir_name(fichier)
    print("pages dans :", page_dir)
    if(os.path.exists(page_dir)):
        sh.rmtree(page_dir)
    os.makedirs(page_dir, exist_ok=True)

    with open(fichier, "r") as f:
        p=0
        lines = []
        for i, line in enumerate(f):
            lines.append(line)
            if (i+1) % nb_tuple_par_page == 0:
                p += 1
                with open(page_dir + f"/page{p}", "w") as fp:
                    fp.write(''.join(lines))
                lines = []
        if len(lines) > 0:
            p +=1
            with open(page_dir + f"/page{p}", "w") as fp:
                fp.write(''.join(lines))
    
        print("nb pages créées :", p)

decoupe_fichier_en_pages(DATA, nb_tuple_par_page=1000)

pages dans : data_pages
nb pages créées : 5000


Afficher le nombre de tuples dans une page (pour quelques pages)

In [6]:
%%bash
wc -l data_pages/* | head -n 3

    1000 data_pages/page1
    1000 data_pages/page10
    1000 data_pages/page100


# Lecture séquentielle du fichier découpé en pages

In [8]:
def lecture_sequentielle_par_page(fichier):
    page_dir = page_dir_name(fichier)
    nb_pages = len(os.listdir(page_dir))

    # a faire : pour chaque page, lire ses lignes
    for ip, page in enumerate(os.listdir(page_dir)):
        # une ligne devient un tuple
        with open(page_dir + "/" + page, 'r') as file:
            for it, line in enumerate(file.readlines()):
            # retourner un itérateur contenant le numéro de page, la position dans la page et le tuple
                yield (ip, it, line)

In [14]:
nb_p = 0
nb_t = 0
last_p = -1
for (ip, it, line) in tqdm.tqdm(lecture_sequentielle_par_page(DATA)):
    nb_t += 1
    if last_p != ip: 
        last_p = ip
        nb_p += 1
print('Noùmbre de pages lues :', nb_p)
print('Nombre de tuples lus :', nb_t)

5000000it [00:05, 947037.65it/s]

Noùmbre de pages lues : 5000
Nombre de tuples lus : 5000000


In [15]:
def filtrer_fichier_par_pages(fichier, valeur_recherchee):
    # a faire pour chaque (numéro de page, position dans la page, tuple) obtnenu en invoquan la méthode ci dessus
    for (ip, it, line) in lecture_sequentielle_par_page(fichier):
        # convertir le 1er attribut en un nombre l'afficher si il est egal à la valeur recherchee
        values = line.split(',')
        att1 = int(values[0])
        if att1 == valeur_recherchee:
            print(att1)

In [17]:
s = np.random.randint(nb_valeurs_distinctes)
print("valeur recherchée :", s)

t1 = time.time()
filtrer_fichier_par_pages(DATA, s)
print("done in", round(time.time() - t1, 2), "s")

valeur recherchée : 72969
72969
done in 6.5 s


# Lecture d'un tuple dans une page

In [30]:
def lecture_tuple(fichier, num_page, position):
    page_dir = page_dir_name(fichier)
    nb_pages = len(os.listdir(page_dir_name(DATA)))
    assert num_page < nb_pages
    page = None
    for i in range(nb_pages):
        if int(os.listdir(page_dir_name(DATA))[i][4:]) == num_page:
            page = os.listdir(page_dir_name(DATA))[i]
            print(page)
            break
    with open(page_dir + "/" + page, 'r') as filep:
        return filep.readlines()[position]

In [32]:
lecture_tuple(DATA, 100, 50)
# 1483400,490295,910293,40978,155996,4719,56866

page100


'1483400,490295,910293,40978,155996,4719,56866,----------------------------------------------------------------------------------------------------\n'

# Créer un index

In [45]:
def creation_index_unique(fichier):
    index = {}
    page_dir = page_dir_name(fichier)
    for page in os.listdir(page_dir):
        ip = int(page[4:])
        with open(page_dir + "/" + page, 'r') as file:
            for it, line in enumerate(file.readlines()):
                index[int(line.split(',')[0])] = (ip, it)
    return index

t1 = time.time()
index1 = creation_index_unique(DATA)
print("done in", round(time.time() - t1, 2), "s")
print('Taille de l index :', len(index1))

done in 7.72 s
Taille de l index : 5000000


# Accès par index

## Index unique scan
Accès pour rechercher les tuples dont le 1er attribut a une valeur donnée.

On peut supposer pour simplifier que l'attribut est unique

In [46]:
def selection_par_index(index, valeur_recherchee):
    return index.get(valeur_recherchee, None)

In [47]:
s = np.random.randint(nb_valeurs_distinctes)
print("valeur recherchée :", s)

ip, it = selection_par_index(index1, s)
print('Page :', ip)
print('Tuple :', it)
print(lecture_tuple(DATA, ip, it))

valeur recherchée : 68978
Page : 2304
Tuple : 625
page2304
68978,2182910,149076,36779,148292,112114,47231,----------------------------------------------------------------------------------------------------



## Index range scan
Accès pour rechercher les tuples dont le 1er attribut a une valeur comprise dans une intervalle donné

In [48]:
def selection_par_index_plage(index, borne_inf, borne_sup):
    rowid = []
    for i in range(borne_inf, borne_sup + 1):
        rowid.append(index.get(i, None))
    return rowid

In [49]:
rowid = selection_par_index_plage(index1, 43514, 43530)
for ri in rowid:
    if ri is None: continue
    print(ri, ':', lecture_tuple(DATA, *ri))

page4226
(4226, 638) : 43514,1288396,631554,528962,17537,95800,49807,----------------------------------------------------------------------------------------------------

page3086
(3086, 209) : 43515,2194131,1128265,160038,179964,150984,11234,----------------------------------------------------------------------------------------------------

page3738
(3738, 919) : 43516,2272190,889051,436988,192265,129457,69934,----------------------------------------------------------------------------------------------------

page4865
(4865, 751) : 43517,2182390,864896,123282,9575,64001,47653,----------------------------------------------------------------------------------------------------

page3293
(3293, 602) : 43518,2086956,135441,249415,109379,55824,76018,----------------------------------------------------------------------------------------------------

page4499
(4499, 301) : 43519,1404955,933659,371681,56688,137967,67596,----------------------------------------------------------------------

# Mise à jour de données




## Sélectionner un tuple et modifier un de ses attributs



In [55]:
ip, it = index1[43530]
page_dir = page_dir_name(DATA)
nb_pages = len(os.listdir(page_dir))
page = None
for i in range(nb_pages):
    if int(os.listdir(page_dir_name(DATA))[i][4:]) == ip:
        page = os.listdir(page_dir_name(DATA))[i]
        print(page)
        break
file = open(page_dir + "/" + page, "r+")
lines = file.readlines()
atts = lines[it].split(',')
atts[0] = '11111111'
lines[it] = ','.join(atts)
file.seek(0)
file.writelines(lines)
file.close()

page2694


##Modifier l'index en conséquence lorsque l'attibut modifié est indexé


In [56]:
index1[11111111] = (ip, it)
del index1[43530]

In [57]:
lecture_tuple(DATA, ip, it)

page2694


'11111111,2338459,788872,448535,119387,122888,7770,----------------------------------------------------------------------------------------------------\n'

# Persistence



## Stocker un index (dans plusieurs pages) pour le reconstruire plus rapidement


In [66]:
def index_dir_name(fichier):
    return fichier.split('.')[0] + "_index"

def decoupe_index_en_pages(fichier, nb_tuple_par_page):
    index_dir = index_dir_name(fichier)
    index = creation_index_unique(fichier)
    print("index dans :", index_dir)
    if(os.path.exists(index_dir)):
        sh.rmtree(index_dir)
    os.makedirs(index_dir, exist_ok=True)
    
    items = list(index.items())
    items.sort()
    nb_i = len(items)
    
    nb_t = 0
    index_n = 0
    for i in range(nb_tuple_par_page, nb_i, nb_tuple_par_page):
        index_n += 1
        a = items[i - nb_tuple_par_page : i]
        lines = list(map(lambda x: ','.join([str(x[0]), str(x[1][0]), str(x[1][1])]) + '\n', a))
        # nouveau fichier
        with open(index_dir + "/index" + str(index_n), 'w') as fi:
            fi.write(''.join(lines))
        nb_t += nb_tuple_par_page
    # prise en compte des index restants -> page non pleine
    if nb_t < nb_i:
        index_n += 1
        a = items[nb_t :]
        lines = list(map(lambda x: ','.join([str(x[0]), str(x[1][0]), str(x[1][1])]) + '\n', a))
        # nouveau fichier
        with open(index_dir + "/index" + str(index_n), 'w') as fi:
            fi.write(''.join(lines))
        nb_t = nb_i
        
    print('Nombre de pages d index :', index_n)

decoupe_index_en_pages(DATA, nb_tuple_par_page=1000)

index dans : data_index
Nombre de pages d index : 5000


## Adapter en conséquence les opérations de modification de l'index

In [81]:
def selection_par_index2(fichier, valeur_recherchee, nb_tuple_par_page=1000):
    index_dir = index_dir_name(fichier)
    index_n = (valeur_recherchee // nb_tuple_par_page) + 1
    
    ip = -1
    it = -1
    with open(index_dir + "/index" + str(index_n), 'r') as fi:
        for line in fi.readlines():
            values = line.split(',')
            key = int(values[0])
            if key == valeur_recherchee:
                ip = int(values[1])
                it = int(values[2]) 
                break
                
    return (ip, it)

In [82]:
s = np.random.randint(nb_valeurs_distinctes)
print("valeur recherchée :", s)

ip, it = selection_par_index2(DATA, s)
print('Page :', ip)
print('Tuple :', it)
if ip != -1:
    print(lecture_tuple(DATA, ip, it))

valeur recherchée : 26925
Page : 1937
Tuple : 986
page1937
26925,621439,918202,426466,61097,129592,20608,----------------------------------------------------------------------------------------------------



In [96]:
def selection_par_index_plage2(fichier, borne_inf, borne_sup, nb_tuple_par_page=1000):
    index_dir = index_dir_name(fichier)
    rowid = []
    
    index_inf = (borne_inf // nb_tuple_par_page) + 1
    index_sup = (borne_sup // nb_tuple_par_page) + 1
    
    for index_n in range(index_inf, index_sup + 1):
        ip = -1
        it = -1
        with open(index_dir + "/index" + str(index_n), 'r') as fi:
            for line in fi.readlines():
                values = line.split(',')
                key = int(values[0])
                if borne_inf <= key <= borne_sup :
                    ip = int(values[1])
                    it = int(values[2])
                    rowid.append((ip, it))
                if key > borne_sup:
                    break
        
    return rowid

In [97]:
rowid = selection_par_index_plage2(DATA, 43514, 43529)
for ri in rowid:
    if ri[0] == -1: continue
    print(ri, ':', lecture_tuple(DATA, *ri))

page4226
(4226, 638) : 43514,1288396,631554,528962,17537,95800,49807,----------------------------------------------------------------------------------------------------

page3086
(3086, 209) : 43515,2194131,1128265,160038,179964,150984,11234,----------------------------------------------------------------------------------------------------

page3738
(3738, 919) : 43516,2272190,889051,436988,192265,129457,69934,----------------------------------------------------------------------------------------------------

page4865
(4865, 751) : 43517,2182390,864896,123282,9575,64001,47653,----------------------------------------------------------------------------------------------------

page3293
(3293, 602) : 43518,2086956,135441,249415,109379,55824,76018,----------------------------------------------------------------------------------------------------

page4499
(4499, 301) : 43519,1404955,933659,371681,56688,137967,67596,----------------------------------------------------------------------